# Executive Compensation Extraction Pipeline

This notebook processes SEC DEF14A filings to extract Summary Compensation Tables using:
1. **MinerU** - PDF to structured data conversion
2. **Qwen3-VL** - Vision-Language Model for table classification and extraction

## 1. Setup & Configuration

In [1]:
%matplotlib inline

# Standard library
import json
import random
import sys
from pathlib import Path

# Third party
import matplotlib.pyplot as plt
from datasets import load_dataset
from IPython.display import display
from openai import AsyncOpenAI
from PIL import Image as PILImage
from tqdm import tqdm

# Local imports
sys.path.insert(0, str(Path("..").resolve()))
from src import (
    convert_docs_to_pdf,
    process_pdfs_with_mineru,
    extract_tables_from_output,
    merge_consecutive_tables,
    find_summary_compensation_in_doc,
    extract_all_summary_compensation,
    save_classification_results,
    save_extraction_results,
    display_all_results,
    display_table_preview,
    TableType,
    fix_all_orphan_images,
)

/home/pdipasquale/MIIA/stuff/.stuff/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# =============================================================================
# CONFIGURATION
# =============================================================================

# Paths
BASE_PATH = Path("..").resolve()
PDF_PATH = BASE_PATH / "pdfs"
OUTPUT_PATH = BASE_PATH / "output"
DATA_PATH = BASE_PATH / "data/DEF14A_all.jsonl"

# Sampling
SEED = 42424242
SAMPLE_SIZE = 150

# VLM Server
VLM_BASE_URL = "http://localhost:8000/v1"
VLM_MODEL = "Qwen/Qwen3-VL-32B-Instruct"

# MinerU
MINERU_MAX_CONCURRENT = 128

# Create directories
PDF_PATH.mkdir(exist_ok=True)
OUTPUT_PATH.mkdir(exist_ok=True)

print(f"Base path: {BASE_PATH}")
print(f"Sample size: {SAMPLE_SIZE} documents")

Base path: /home/pdipasquale/MIIA/stuff
Sample size: 150 documents


In [3]:
# Load dataset
dataset = load_dataset("json", data_files=str(DATA_PATH))
all_docs = dataset["train"]

# Sample documents
random.seed(SEED)
indices = random.sample(range(len(all_docs)), SAMPLE_SIZE)
docs = all_docs.select(indices)

# Initialize VLM client
client = AsyncOpenAI(base_url=VLM_BASE_URL, api_key="dummy")

print(f"Loaded {len(all_docs):,} documents, sampled {len(docs)}")

Loaded 109,087 documents, sampled 150


## 2. PDF Processing & Table Extraction

Convert HTML documents to PDF, process with MinerU, and extract tables.

In [5]:
# Step 1: Convert HTML to PDF
convert_docs_to_pdf(docs, base_path=BASE_PATH)

# Step 2: Process PDFs with MinerU (VLM backend)
failed, success = process_pdfs_with_mineru(base_path=BASE_PATH, max_concurrent=MINERU_MAX_CONCURRENT)
print(f"\n✓ MinerU: {len(success)} successful, {len(failed)} failed")

100%|██████████| 150/150 [00:50<00:00,  2.94it/s]


Found 150 PDFs total
  - Already processed (skipped): 100
  - To process: 50

Processing 50 PDFs (max 128 concurrent)


100%|██████████| 50/50 [09:36<00:00, 11.52s/it]  


=== Processing Complete ===
Success: 50
Failed: 0

✓ MinerU: 50 successful, 0 failed


In [7]:
# Step 3: Fix orphan images (MinerU bug - images exist but not linked in JSON)
print("Fixing orphan images...")
fix_stats = fix_all_orphan_images(OUTPUT_PATH)
print(f"✓ Fixed {fix_stats['total_fixed']} tables in {fix_stats['docs_fixed']} documents")

# Step 4: Extract all tables from MinerU output
all_tables, extraction_stats = extract_tables_from_output(
    output_path=OUTPUT_PATH, 
    save_path=str(BASE_PATH / "all_tables.json")
)
print(f"✓ Extracted {len(all_tables)} tables from {len(extraction_stats['with_tables'])} documents")

Fixing orphan images...


✓ Fixed 0 tables in 0 documents
=== Extraction Summary ===
Total output directories: 150
  - MinerU processed: 150
  - MinerU NOT processed (no content_list.json): 0

Of processed documents:
  - With tables: 150
  - Without tables: 0

Total tables extracted: 3129
✓ Extracted 3129 tables from 150 documents


In [8]:
# Load tables (if restarting from here)
with open(BASE_PATH / "all_tables.json") as f:
    all_tables = json.load(f)

# Statistics
doc_sources = set(t.get('source_doc') for t in all_tables)
all_output_dirs = {d.name for d in OUTPUT_PATH.iterdir() if d.is_dir()}
docs_without_tables = all_output_dirs - doc_sources

print(f"Documents with tables: {len(doc_sources)}")
print(f"Documents without tables: {len(docs_without_tables)}")
print(f"Total tables: {len(all_tables)}")

Documents with tables: 150
Documents without tables: 0
Total tables: 3129


## 3. Single Document Processing (Debug)

Test classification and extraction on a single document.

In [10]:
# Select document by index
DOC_INDEX = 35

doc_sources = list(set(t.get('source_doc') for t in all_tables))
source_doc = doc_sources[DOC_INDEX]

# Load metadata
metadata_path = OUTPUT_PATH / source_doc / "metadata.json"
with open(metadata_path) as f:
    meta = json.load(f)

# Check if it's a fund (no SIC code)
if meta.get("sic") in ("NULL", None):
    print("⚠️  This is a FUND (no SIC code) - skipping")
else:
    print(f"Document: {source_doc}")
    print(f"Company: {meta.get('company_name', 'N/A')}")
    print(f"SIC: {meta.get('sic', 'N/A')}")

Document: 1326428_2006_0000950134-06-023257
Company: N/A
SIC: 1311


In [ ]:
# Classify tables in document
found, all_classifications = await find_summary_compensation_in_doc(
    doc_source=source_doc,
    all_tables=all_tables,
    client=client,
    model=VLM_MODEL,
    base_path=BASE_PATH,
    display_func=display,
    plt_module=plt,
    pil_image_class=PILImage,
    debug=False
)


# Merge consecutive tables if needed
if found:
    images_base_dir = OUTPUT_PATH / source_doc / source_doc / "vlm"
    found = merge_consecutive_tables(found, images_base_dir, all_tables, all_classifications, debug=True)

Found 1 Summary Compensation Tables
Found 1 Summary Compensation table(s)


In [ ]:
# Extract compensation data
extracted = await extract_all_summary_compensation(
    found_tables=found,
    all_tables=all_tables,
    client=client,
    model=VLM_MODEL,
    base_path=BASE_PATH,
    metadata=meta
)

# Save results
save_classification_results(found, OUTPUT_PATH / source_doc, metadata=meta)
save_extraction_results(extracted, OUTPUT_PATH / source_doc, metadata=meta)

# Display results
display_all_results(
    extracted_list=extracted,
    found_tables=found,
    metadata=meta,
    base_path=BASE_PATH,
    pil_image_class=PILImage
)

## 4. Batch Processing

Process all documents. Skips funds (no SIC) and already processed documents.

In [14]:
doc_sources = list(set(t.get('source_doc') for t in all_tables))

# Track statistics
stats = {"processed": 0, "skipped_fund": 0, "skipped_done": 0, "no_tables": 0, "errors": 0}
no_sct_docs = []  # Lista documenti senza SCT

for source_doc in tqdm(doc_sources, desc="Processing documents"):
    # Load metadata
    metadata_path = OUTPUT_PATH / source_doc / "metadata.json"
    with open(metadata_path) as f:
        meta = json.load(f)
    
    # Skip funds (no SIC code)
    if meta.get("sic") in ("NULL", None):
        stats["skipped_fund"] += 1
        continue
    
    # Skip if already processed
    if (OUTPUT_PATH / source_doc / "extraction_results.json").exists():
        stats["skipped_done"] += 1
        continue
    
    try:
        # Step 1: Classify tables
        found, all_classifications = await find_summary_compensation_in_doc(
            doc_source=source_doc,
            all_tables=all_tables,
            client=client,
            model=VLM_MODEL,
            base_path=BASE_PATH,
            display_func=display,
            plt_module=plt,
            pil_image_class=PILImage,
            debug=False
        )
        
        if not found:
            stats["no_tables"] += 1
            no_sct_docs.append({"doc": source_doc, "link": meta.get("filing_html_index", "N/A")})
            tqdm.write(f"⚠️ No SCT: {source_doc}")
            tqdm.write(f"   📎 {meta.get('filing_html_index', 'N/A')}")
            continue
        
        # Step 2: Merge consecutive tables
        images_base_dir = OUTPUT_PATH / source_doc / source_doc / "vlm"
        found = merge_consecutive_tables(found, images_base_dir, all_tables, all_classifications, debug=False)
        
        # Step 3: Extract compensation data
        extracted = await extract_all_summary_compensation(
            found_tables=found,
            all_tables=all_tables,
            client=client,
            model=VLM_MODEL,
            base_path=BASE_PATH,
            metadata=meta
        )
        
        # Step 4: Save results
        save_classification_results(found, OUTPUT_PATH / source_doc, metadata=meta)
        save_extraction_results(extracted, OUTPUT_PATH / source_doc, metadata=meta)
        stats["processed"] += 1
        
    except Exception as e:
        stats["errors"] += 1
        tqdm.write(f"✗ Error {source_doc}: {e}")

# Print summary
print("\n" + "="*60)
print("PROCESSING COMPLETE")
print("="*60)
print(f"Processed:      {stats['processed']}")
print(f"No SCT found:   {stats['no_tables']}")
print(f"Skipped (fund): {stats['skipped_fund']}")
print(f"Skipped (done): {stats['skipped_done']}")
print(f"Errors:         {stats['errors']}")

if no_sct_docs:
    print(f"\n📋 Documents without SCT:")
    for item in no_sct_docs:
        print(f"  - {item['doc']}")
        print(f"    {item['link']}")

Processing documents:   7%|▋         | 11/150 [00:07<01:38,  1.41it/s]

Found 0 Summary Compensation Tables
⚠️ No SCT: 1446371_2020_0001446371-20-000021
   📎 https://www.sec.gov/Archives/edgar/data/1446371/0001446371-20-000021-index.html


Processing documents:  11%|█▏        | 17/150 [00:21<03:02,  1.37s/it]

Found 0 Summary Compensation Tables
⚠️ No SCT: 1453090_2014_0001308179-14-000176
   📎 https://www.sec.gov/Archives/edgar/data/1453090/0001308179-14-000176-index.html


Processing documents:  13%|█▎        | 20/150 [00:35<04:41,  2.17s/it]

...

✓ Extracted table 30: 16 executives
💾 Classification results saved to: /home/pdipasquale/MIIA/stuff/output/764478_2019_0001140361-19-008156/classification_results.json
💾 Extraction results saved to: /home/pdipasquale/MIIA/stuff/output/764478_2019_0001140361-19-008156/extraction_results.json
Found 1 Summary Compensation Tables


Processing documents:  99%|█████████▉| 149/150 [1:19:31<01:01, 61.15s/it]

✓ Extracted table 9: 12 executives
💾 Classification results saved to: /home/pdipasquale/MIIA/stuff/output/1358356_2020_0001157523-20-000586/classification_results.json
💾 Extraction results saved to: /home/pdipasquale/MIIA/stuff/output/1358356_2020_0001157523-20-000586/extraction_results.json
Found 1 Summary Compensation Tables


Processing documents: 100%|██████████| 150/150 [1:19:47<00:00, 31.92s/it]

✓ Extracted table 4: 2 executives
💾 Classification results saved to: /home/pdipasquale/MIIA/stuff/output/81318_2011_0001144204-11-024056/classification_results.json
💾 Extraction results saved to: /home/pdipasquale/MIIA/stuff/output/81318_2011_0001144204-11-024056/extraction_results.json

PROCESSING COMPLETE
Processed:      90
No SCT found:   4
Skipped (fund): 31
Skipped (done): 25
Errors:         0

📋 Documents without SCT:
  - 1446371_2020_0001446371-20-000021
    https://www.sec.gov/Archives/edgar/data/1446371/0001446371-20-000021-index.html
  - 1453090_2014_0001308179-14-000176
    https://www.sec.gov/Archives/edgar/data/1453090/0001308179-14-000176-index.html
  - 47307_2012_0000047307-12-000022
    https://www.sec.gov/Archives/edgar/data/47307/0000047307-12-000022-index.html
  - 1446159_2019_0001171843-19-006199
    https://www.sec.gov/Archives/edgar/data/1446159/0001171843-19-006199-index.html


## 5. Create HuggingFace Dataset

Build a dataset from extraction results with table images.

In [15]:
from datasets import Dataset, Features, Value, Image as HFImage, Sequence
from huggingface_hub import HfApi

# Trova tutti i documenti con extraction_results.json
records = []

for doc_dir in OUTPUT_PATH.iterdir():
    if not doc_dir.is_dir():
        continue
    
    extraction_file = doc_dir / "extraction_results.json"
    classification_file = doc_dir / "classification_results.json"
    metadata_file = doc_dir / "metadata.json"
    
    # Carica metadata
    if not metadata_file.exists():
        continue
    with open(metadata_file) as f:
        meta = json.load(f)
    
    # Escludi fondi (no SIC)
    if meta.get("sic") == "NULL" or meta.get("sic") is None:
        continue
    
    # Se non ha extraction results, skip
    if not extraction_file.exists() or not classification_file.exists():
        continue
    
    with open(extraction_file) as f:
        extraction = json.load(f)
    with open(classification_file) as f:
        classification = json.load(f)
    
    # Base record con metadati
    base_record = {
        "cik": meta.get("cik"),
        "company": meta.get("company"),
        "year": meta.get("year"),
        "filing_date": meta.get("filing_date"),
        "sic": meta.get("sic"),
        "state_of_inc": meta.get("state_of_inc"),
        "filing_html_index": meta.get("filing_html_index"),
        "accession_number": meta.get("accession_number"),
    }
    
    # Per ogni tabella trovata
    for i, table_info in enumerate(classification.get("tables", [])):
        record = base_record.copy()
        
        # Immagine
        img_path = table_info.get("table", {}).get("img_path", "")
        images_dir = doc_dir / doc_dir.name / "vlm"
        full_img_path = images_dir / img_path
        
        if full_img_path.exists():
            record["table_image"] = str(full_img_path)
        else:
            record["table_image"] = None
        
        # HTML body
        record["table_body"] = table_info.get("table", {}).get("table_body", "")
        
        # Executives (dalla extraction corrispondente)
        if i < len(extraction.get("data", [])):
            execs = extraction["data"][i].get("executives", [])
            record["executives"] = json.dumps(execs)
        else:
            record["executives"] = json.dumps([])
        
        records.append(record)

print(f"Total records: {len(records)}")

Total records: 128


In [ ]:
# Crea dataset HuggingFace
hf_dataset = Dataset.from_list(records)

# Cast immagini
hf_dataset = hf_dataset.cast_column("table_image", HFImage())

print(hf_dataset)
print(hf_dataset[0])

In [ ]:
# Save dataset locally
hf_dataset.save_to_disk(str(BASE_PATH / "hf/execcomp-ai-sample"))
print(f"✓ Dataset saved to {BASE_PATH / 'hf/execcomp-ai-sample'}")

In [ ]:
# =============================================================================
# UPLOAD TO HUGGING FACE HUB
# =============================================================================
# Uncomment the following lines to upload the dataset to HuggingFace Hub
# Make sure you're logged in: huggingface-cli login

# api = HfApi()
# hf_dataset.push_to_hub("pierjoe/execcomp-ai-sample")